# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


In [2]:
!pip3 install -U sagemaker

  Using cached sagemaker-2.239.0-py3-none-any.whl.metadata (16 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached s3transfer-0.11.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
Using cached sagemaker-2.239.0-py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.4 MB/s eta 0:00:00ta 0:00:01
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.9/404.9 kB 4.1 MB/s eta 0:00:00:00:01
Using cached uvicorn-0.34.0-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 34.5 MB/s eta 0:00:00:00:010:01
Using cached mock-4.0.3-py3-none-any.whl (28 kB)
Using cached s3transfer-0.11.2-py3-none-any.whl (84 kB)
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    U

In [3]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "flight-delay-prediction-xgboost"

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Enable inline plots in Jupyter
%matplotlib inline

print("Libraries imported successfully.")

Libraries imported successfully.


In [7]:
# Cell [2]: Load the dataset from S3
%store -r s3_private_path_tsv  # retrieves the S3 path variable stored in earlier notebooks

# Construct the full S3 CSV path
s3_csv_path = f"{s3_private_path_tsv}/Airline_Delay_Cause.csv.gz"
print("Loading data from:", s3_csv_path)

# Read the CSV file, assuming it is compressed using gzip and has a header row
df = pd.read_csv(s3_csv_path, compression='gzip')

# Print basic information
print("Data shape:", df.shape)
df.head()

no stored variable or alias #
no stored variable or alias retrieves
no stored variable or alias the
no stored variable or alias S3
no stored variable or alias path
no stored variable or alias variable
no stored variable or alias stored
no stored variable or alias in
no stored variable or alias earlier
no stored variable or alias notebooks
Loading data from: s3://sagemaker-us-east-1-546928460657/airline-delay-cause/csv/Airline_Delay_Cause.csv.gz


/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


[02/07/25 16:40:42] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=721328;file:///opt/conda/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=313646;file:///opt/conda/lib/python3.10/site-packages/botocore/httpchecksum.py#487\487]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

[02/07/25 16:40:43] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=832025;file:///opt/conda/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=198485;file:///opt/conda/lib/python3.10/site-packages/botocore/httpchecksum.py#487\487]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

Data shape: (390637, 19)


,year,month,carrier,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,ABE,81.0,7.0,5.61,0.56,0.84,0.0,0.00,0.0,0.0,243.0,201.0,15.0,27.0,0.0,0.0
1,2024,9,9E,AEX,81.0,6.0,4.43,0.00,1.13,0.0,0.44,0.0,0.0,647.0,440.0,0.0,189.0,0.0,18.0
2,2024,9,9E,AGS,133.0,12.0,3.49,2.04,3.84,0.0,2.63,10.0,0.0,2382.0,1269.0,594.0,167.0,0.0,352.0
3,2024,9,9E,ALB,73.0,4.0,0.82,1.00,0.23,0.0,1.95,0.0,0.0,160.0,28.0,15.0,9.0,0.0,108.0
4,2024,9,9E,ATL,2204.0,280.0,61.70,17.36,88.36,0.0,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0


In [14]:
# Add target column: ArrivalDelayed
# A flight is considered delayed if arr_del15 >= 1 (15+ min late)
df['ArrivalDelayed'] = df['arr_del15'].apply(lambda x: 1 if x >= 15 else 0)
df.head()

,year,month,carrier,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,ArrivalDelayed
0,2024,9,9E,ABE,81.0,7.0,5.61,0.56,0.84,0.0,0.00,0.0,0.0,243.0,201.0,15.0,27.0,0.0,0.0,0
1,2024,9,9E,AEX,81.0,6.0,4.43,0.00,1.13,0.0,0.44,0.0,0.0,647.0,440.0,0.0,189.0,0.0,18.0,0
2,2024,9,9E,AGS,133.0,12.0,3.49,2.04,3.84,0.0,2.63,10.0,0.0,2382.0,1269.0,594.0,167.0,0.0,352.0,0
3,2024,9,9E,ALB,73.0,4.0,0.82,1.00,0.23,0.0,1.95,0.0,0.0,160.0,28.0,15.0,9.0,0.0,108.0,0
4,2024,9,9E,ATL,2204.0,280.0,61.70,17.36,88.36,0.0,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0,1


In [15]:
# Encode categorical features
from sklearn.preprocessing import LabelEncoder

for col in ['carrier', 'airport']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [23]:
df.fillna(0, inplace=True)  # Handle missing values

# Split data into development and production sets
dev_df = df[(df['year'] >= 2004) & (df['year'] <= 2015)]
prod_df = df[(df['year'] >= 2016) & (df['year'] <= 2024)]

In [24]:
# Save development and production data
dev_csv_path = "development_data.csv"
prod_csv_path = "production_data.csv"

dev_df.to_csv(dev_csv_path, index=False)
prod_df.to_csv(prod_csv_path, index=False)

# Upload development and production data to S3
dev_s3_path = sess.upload_data(path=dev_csv_path, bucket=bucket, key_prefix=prefix+"/development")
prod_s3_path = sess.upload_data(path=prod_csv_path, bucket=bucket, key_prefix=prefix+"/production")


In [25]:
# Split production data into training & validation sets
from sklearn.model_selection import train_test_split

X = dev_df.drop('ArrivalDelayed', axis=1)
y = dev_df['ArrivalDelayed']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to CSV for SageMaker (label column must be first)
train_combined = pd.concat([y_train, X_train], axis=1)
val_combined = pd.concat([y_val, X_val], axis=1)

train_csv_path = "train.csv"
val_csv_path = "validation.csv"

train_combined.to_csv(train_csv_path, index=False, header=False)
val_combined.to_csv(val_csv_path, index=False, header=False)

# Upload data to S3
train_s3_path = sess.upload_data(path=train_csv_path, bucket=bucket, key_prefix=prefix+"/train")
val_s3_path = sess.upload_data(path=val_csv_path, bucket=bucket, key_prefix=prefix+"/validation")

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [26]:
# Retrieve XGBoost container image
from sagemaker.image_uris import retrieve
from sagemaker.inputs import TrainingInput
xgboost_image_uri = retrieve("xgboost", region, "1.5-1")

# Define XGBoost estimator
xgb = sagemaker.estimator.Estimator(
    image_uri=xgboost_image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=sess
)

xgb.set_hyperparameters(
    objective="binary:logistic",
    num_round=100,
    max_depth=5,
    eta=0.2,
    subsample=0.8,
    eval_metric="auc"
)

# Define training inputs
train_input = TrainingInput(train_s3_path, content_type="text/csv")
val_input = TrainingInput(val_s3_path, content_type="text/csv")

# Train the model
xgb.fit({"train": train_input, "validation": val_input})

[02/07/25 18:19:23] INFO     Ignoring unnecessary instance type: None.                            ]8;id=851054;file:///opt/conda/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=282466;file:///opt/conda/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=84203;file:///opt/conda/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=42662;file:///opt/conda/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=938900;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=220781;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-02-07-18-19-23-604                                             

2025-02-07 18:19:25 Starting - Starting the training job...
2025-02-07 18:19:40 Starting - Preparing the instances for training...
2025-02-07 18:20:07 Downloading - Downloading input data...
2025-02-07 18:20:33 Downloading - Downloading the training image...
2025-02-07 18:21:13 Training - Training image download completed. Training in progress./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-02-07 18:21:21.411 ip-10-0-185-250.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-02-07 18:21:21.432 ip-10-0-185-250.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-02-07:18:21:21:INFO] Imported framework sagemaker_xgboost_container.training
[2025-02-07:18:21:21:INFO] Failed to parse hyperparameter eval_metric value auc to

In [21]:
# Deploy the model as a SageMaker endpoint
endpoint_name = "flight-delay-xgboost-endpoint"
predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name
)

[02/07/25 17:47:43] INFO     Creating model with name: sagemaker-xgboost-2025-02-07-17-47-42-946    ]8;id=708185;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=688813;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name flight-delay-xgboost-endpoint       ]8;id=775670;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=756612;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

[02/07/25 17:47:44] INFO     Creating endpoint with name flight-delay-xgboost-endpoint              ]8;id=151175;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=724312;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!

---

## Batch Transform



In [33]:
# Only the features (no label)
small_test_set = prod_df.drop('ArrivalDelayed', axis=1).sample(500)

small_test_csv_path = "small_test.csv"
small_test_set.to_csv(small_test_csv_path, index=False, header=False)

# Upload smaller dataset to S3
small_test_s3_path = sess.upload_data(
    path=small_test_csv_path,
    bucket=bucket,
    key_prefix=prefix + "/small_test"
)


In [34]:
batch_input_s3 = small_test_s3_path
batch_output_s3 = f"s3://{bucket}/{prefix}/batch-output"

transformer = xgb.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=batch_output_s3
)

transformer.transform(
    data=batch_input_s3,
    content_type="text/csv",
    split_type="Line"
)
transformer.wait()

print("Batch transform job complete.")

[02/07/25 19:38:07] INFO     Creating model with name: sagemaker-xgboost-2025-02-07-19-38-07-467    ]8;id=9389;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=243363;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[02/07/25 19:38:08] INFO     Creating transform job with name:                                      ]8;id=310752;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=95956;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#3951\3951]8;;\
                             sagemaker-xgboost-2025-02-07-19-38-08-268                                             

........................./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-02-07:19:42:15:INFO] No GPUs detected (normal if no gpus installed)
[2025-02-07:19:42:15:INFO] No GPUs detected (normal if no gpus installed)
[2025-02-07:19:42:15:INFO] nginx config: 
worker_processes auto;
/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-02-07:19:42:15:INFO] No GPUs detected (normal if no gpus installed)
[2025-02-07:19:42:15:INFO] No GPUs detected (normal if no gpus installed)
[2025-02-07:19:42:15:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;

In [38]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name_batch_transform = "flight-delay-xgboost-endpoint-with-batch-transform"

predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name_batch_transform,
    serializer=CSVSerializer(),              # <--- important
    deserializer=JSONDeserializer()          # or StringDeserializer() depending on your output
)


[02/07/25 19:45:32] INFO     Creating model with name: sagemaker-xgboost-2025-02-07-19-45-32-217    ]8;id=921555;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=333863;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[02/07/25 19:45:33] INFO     Creating endpoint-config with name                                     ]8;id=399556;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=879648;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             flight-delay-xgboost-endpoint-with-batch-transform                                    

                    INFO     Creating endpoint with name                                            ]8;id=144232;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=912031;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             flight-delay-xgboost-endpoint-with-batch-transform                                    

------!

In [39]:
# Test inference with valid data (multiple samples)
test_samples = X_val.sample(5).values  # Select 5 random validation samples
import io
csv_buffer = io.StringIO()
np.savetxt(csv_buffer, test_samples, delimiter=",")
test_samples_csv = csv_buffer.getvalue().strip()

response = predictor.predict(test_samples_csv)
print("Predicted probabilities of delay for test samples:", response)

# Cleanup: Uncomment to delete endpoint when done
# predictor.delete_endpoint()

Predicted probabilities of delay for test samples: {'predictions': [{'score': 0.9999898672103882}, {'score': 0.9999855756759644}, {'score': 0.9999898672103882}, {'score': 0.9999898672103882}, {'score': 0.9999898672103882}]}


In [41]:
# Cleanup: Uncomment to delete endpoint when done
predictor.delete_endpoint()

[02/07/25 19:52:15] INFO     Deleting endpoint configuration with name:                             ]8;id=962741;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=674510;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\
                             flight-delay-xgboost-endpoint-with-batch-transform                                    

                    INFO     Deleting endpoint with name:                                           ]8;id=55380;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=983775;file:///opt/conda/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\
                             flight-delay-xgboost-endpoint-with-batch-transform                                    